In [1]:
#import modules
import json
from binance.client import Client
import pandas as pd
from envs import env
from dotenv import load_dotenv

In [2]:
#import classes from ./ folder
import timescaledbAccess

In [3]:
#read env vars
load_dotenv()

True

In [4]:
timescale = timescaledbAccess.timescaleAccess()
try:
    liveVolume=env("liveVolume")
    apiSecret=env('apiSecret')
    apiKey=env('apiKey')
except KeyError:
    print("No env variables set.")
    sys.exit(1)
#connect to binance to get current balance
client = Client(apiKey, apiSecret, {'timeout':600})

In [5]:
sql = ("SELECT id, askprice, managedassets" +
    " FROM table001 WHERE" +
    " resultpercent IS NULL " +
    " AND takeprofit IS NOT NULL;")
bA = pd.DataFrame(timescale.sqlQuery(sql))
bA = bA.apply(pd.to_numeric, errors='coerce')

In [6]:
print(bA)

       0       1        2
0  76024  0.1592  0.00042


In [10]:
sql = ("select symbol from table001 where id = '" + str(bA[0][0]) + "';")
symbol = pd.DataFrame(timescale.sqlQuery(sql))
sql = ("select max(id) from table001 where symbol = '" + str(symbol[0][0]) + "';")
maxId = pd.DataFrame(timescale.sqlQuery(sql))

In [11]:
print(maxId)

        0
0  115240


In [12]:
percentChange = (float(client.get_asset_balance(asset='BNB')['free']) - bA[2][0]) / 0.1

In [13]:
print(percentChange)

-0.0041979999999999995


In [16]:
sql = ("UPDATE table001 SET" +
" resultpercent = '" + str(percentChange) +
"', stopid = '" + str(maxId[0][0]) +
"' WHERE id = '" + str(maxId[0][0]) +
"';")

In [38]:
percentChange = (float(client.get_asset_balance(asset='BNB')['free']) - bA[2][0]) / 0.1
#get max id
sql = ("select symbol from table001 where id = '" + str(bA[0][0]) + "';")
print(sql)
symbol = pd.DataFrame(timescale.sqlQuery(sql))
print(symbol)

select symbol from table001 where id = '76024';
error in query


UnboundLocalError: local variable 'result' referenced before assignment

In [36]:
sql = ("select max(id) from table001 where symbol = '" + str(symbol[0][0]) + "';")
maxId = pd.DataFrame(timescale.sqlQuery(sql))
sql = ("UPDATE table001 SET" +
" resultpercent = '" + str(percentChange) +
"', stopid = '" + str(maxId) +
"' WHERE id = '" + str(maxId[0][0]) +
"';")
print(sql)
timescale.sqlUpdate(sql)
timescale.databaseClose()

UPDATE table001 SET resultpercent = '-0.0041979999999999995', stopid = '        0
0  113074' WHERE id = '113074';
error in update


In [42]:
sql = ("select count(*) from table001;")
maxId = pd.DataFrame(timescale.sqlQuery(sql))

error in query


UnboundLocalError: local variable 'result' referenced before assignment

In [40]:
timescale.databaseClose()